# SpotiScry
### Date: January 2021
### Project Goal:
### Creators:

## Table of Contents
- [Acquire](#acquire)
- [Prepare](#prepare)
- [Explore](#explore)
- [Model](#model)
- [Conclusions](#conclusions)

# Acquire <a class="anchor" id="acquire"></a>

## Takeaways

# Prepare <a class="anchor" id="prepare"></a>

## Takeaways

# Explore <a class="anchor" id="explore"></a>

## Takeaways

# Model <a class="anchor" id="model"></a>

## Takeaways

# Conclusions <a class="anchor" id="conclusions"></a>